# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [1]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
 

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [8]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

22.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

Внизу представлен набор из 5 тестов, кроме него тестирование проводилось с использованием генерации случайных функций.

In [5]:
# ваш код
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number
import numpy
from scipy.misc import derivative
import math

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(self.value - float(other), self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)

    def __neg__(self) -> "Dual":
        return Dual(-self.value, -self.d)
    
    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        try:
            match other:
                case Dual(o_value, o_d):
                    return Dual(self.value / o_value, (self.d * o_value - o_d * self.value) / o_value**2)
                case Number():
                    return Dual(self.value / float(other), self.d / float(other))
        except ZeroDivisionError:
            print("Деление на ноль")
            exit(-1)


    def __pow__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value ** o_value, self.value ** o_value * (other*log(self)).d)
            case Number():
                return Dual(self.value ** float(other), float(other)*self.value**(other - 1))
    
    def __abs__(self) -> "Dual":
        return Dual(abs(self.value), self.d * numpy.sign(self.value))

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__

    def __rpow__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value ** o_value, self.value ** o_value * (other*log(self)).d)
            case Number():
                    return Dual(float(other) ** self.value, float(other)**self.value*log(float(other)))
                

    def __rtruediv__(self, other: Union["Dual", Number]) -> "Dual":
        try:
            match other:
                case Dual(o_value, o_d):
                    return Dual(self.value / o_value, (self.d * o_value - o_d * self.value) / o_value**2)
                case Number():
                    return Dual(float(other) / self.value, -self.d * float(other) / self.value**2)
        except ZeroDivisionError:
            print("Деление на ноль")
            exit(-1)

    def __rsub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(float(other) - self.value, -self.d)

def log(arg: Union["Dual", Number]) -> "Dual":
    try:
        match arg:
            case Dual(arg.value, arg.d):
                return Dual(math.log(arg.value), arg.d/arg.value)
            case Number():
                return math.log(arg)
    except ValueError:
        print("Выражение под логарифмом <0")
        exit(-1)

def exp(arg: Union["Dual", Number]) -> "Dual":
    match arg:
        case Dual(arg.value, arg.d):
            return Dual(math.exp(arg.value), math.exp(arg.value) * arg.d)
        case Number():
            return math.exp(arg)

def cos(arg: Union["Dual", Number]) -> "Dual":
    match arg:
        case Dual(arg.value, arg.d):
            return Dual(math.cos(arg.value), -math.sin(arg.value)*arg.d)
        case Number():
            return math.cos(arg)

def sin(arg: Union["Dual", Number]) -> "Dual":
    match arg:
        case Dual(arg.value, arg.d):
            return Dual(math.sin(arg.value), math.cos(arg.value)*arg.d)
        case Number():
            return math.sin(arg)

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    def newfunc(*args):
        answers = []
        for j in range(0, len(args)):
            ans = func(*[Dual(args[i], float(i==j)) for i in range(0, len(args))])
            if type(ans) != Dual:
                ans = 0
            else:
                ans = ans.d
            answers.append(ans)
        return answers
    return newfunc
        
# return lambda *args: [func(*[Dual(args[i], float(i==j)) for i in range(0, len(args))]).d for j in range(0, len(args))]

# Функция, которую будем дифференцировать
# def f(x0: float) -> float:
#     return 567-exp(x0)**exp(x0)

f = lambda x0:567-exp(x0)**exp(x0)
f1 = lambda x: x**2 + 2**x
f2 = lambda x: cos(x) + sin(2*x + x**2)
f3 = lambda x: log(8*x - x**2/2)
f4 = lambda x: (5*x**2 + 6*x - 2)/(x**2 + 2*x - cos(x))
f5 = lambda x: exp(x) - exp(cos(x)) - sin(2**x)
f_diff = diff(f)
f_diff1 = diff(f1)
f_diff2 = diff(f2)
f_diff3 = diff(f3)
f_diff4 = diff(f4)
f_diff5 = diff(f5)


# значение производной в точке x = 2
print(*f_diff(2.) - derivative(f, 2., dx=1e-6))
print(*f_diff1(2.) - derivative(f1, 2., dx=1e-6))
print(*f_diff2(2.) - derivative(f2, 2., dx=1e-6))
print(*f_diff3(2.) - derivative(f3, 2., dx=1e-6))
print(*f_diff4(2.) - derivative(f4, 2., dx=1e-6))
print(*f_diff5(2.) - derivative(f5, 2., dx=1e-6))



0.005900658667087555
-4.1588066324038664e-11
2.702504886542556e-11
-2.1850815601354157e-10
4.448025281433843e-12
-3.547278026871936e-10


## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

Сделано выше

## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

Ниже реализован метод односторонней разности.
Функция `derivative` использовалась для тестирования выше

In [7]:
from scipy.misc import derivative

def der(f: Callable[[float], float], x: float, h: float) -> float: # Метод односторонней разности
    return (f(x + h) - f(x))/h

print(der(f, 2., 1e-8))
print(derivative(f, 2., dx=1e-6))

-1.3072872229180632
-1.3072872417363435


## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

Ниже представлена реализация генерации функций в текстовом виде и использование функции `eval`, есть возможность генерировать функции нескольких аргументов.

In [35]:
# ваш код
import random
vars = 3
variables = [f"x{i}" for i in range(0, vars)]
const_max = 1e3
const_min = -1e3
functions = ["cos", "log", "exp", "sin", "abs"]
operands = ["*", "/", "+", "-", "**"]
global_expression = "lambda " + ','.join(variables) + ":"
print(global_expression)  
# функция - random.choice(functions)
# операция - random.choice(operands)
# глубина N = random.randint(a, b), a <= N <= b
# генерируем количество операций
counter = 0
def generate_function():
    global counter
    expression = ""
    ops = random.randint(0, 3)
    skip_left = False
    if counter > 3:
        expression += random.choice(variables)
        return expression
    if ops == 0:
        # например y = sin(log(x + 2))
        function = random.choice(functions + variables*2 + ["const"])
        if function == "const":
            expression += str(random.randint(const_min, const_max))
        elif function in variables:
            expression += function
        else:
            counter +=1
            expression += function + "(" + generate_function() + ")"
    else:
        for _ in range(ops):
            op = random.choice(operands)
            left = random.choice(functions + variables + ["const"])
            right = random.choice(functions + variables + ["const"])
            if not skip_left:
                if left == "const":
                    expression += str(random.randint(const_min, const_max))
                elif left in variables:
                    expression += left
                else:
                    counter +=1
                    expression += left + "(" + generate_function() + ")"
            expression += op
            if right == "const":
                expression += str(random.randint(const_min, const_max))
            elif right in variables:
                 expression += right
            else:
                counter +=1
                expression += right + "(" + generate_function() + ")"
            skip_left = True
    return expression

global_expression += generate_function()
print(global_expression)
func = eval(global_expression)
func_diff = diff(func)
print(func_diff(2., 2., 2.))
#print(derivative(func, 2., dx=1e-6))


lambda x0,x1,x2:
lambda x0,x1,x2:x1
[0.0, 1.0, 0.0]


## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

Реализовано в первом задании. Ниже представлен тест на описанной здесь функции. Также работат протестирована с помощью генератора случайных функций.

In [17]:

# Функция, которую будем дифференцировать
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

# значение производной в точке x = 2
# print(derivative(f, , dx=1e-6))
f_diff(10, 10, 10) # = [10, 5, 1]

[10.0, 5.0, 1.0]